In [ ]:
#imports
import pandas as pd
import matplotlib.pyplot as plt
import folium
from folium.plugins import HeatMap,HeatMapWithTime
import matplotlib.dates as mdates

In [ ]:
data = pd.read_excel('202504-deceleration.xlsx')
data = data.reset_index(drop=True)
data['Date'] = pd.to_datetime(data['Date']).dt.date
pd.set_option('display.max_rows', 10)
data.style

In [ ]:
# First define some useful variables
treshold_dec = data['Deceleration'].median()
min_dec = data['Deceleration'].min()
max_dec = data['Deceleration'].max()
print(treshold_dec)
print(min_dec)
print(max_dec)

map_center = [34.547494239393636, 135.4963598625541] # Goryoyama Park
sw = data[['Latitude', 'Longitude']].min().values.tolist()
ne = data[['Latitude', 'Longitude']].max().values.tolist()

In [ ]:
# Compare heatmaps of Weekdays and Holidays
subdata1 = data[['Latitude','Longitude','Weekday/Holiday']]
subdata1_week = subdata1[subdata1['Weekday/Holiday']=='Weekday'].drop('Weekday/Holiday', axis=1)
subdata1_holi = subdata1[subdata1['Weekday/Holiday']=='Holiday'].drop('Weekday/Holiday', axis=1)

week_map = folium.Map(location=map_center,tiles='openstreetmap',zoomSnap=0.5)
HeatMap(subdata1_week,radius=14).add_to(week_map)
week_map.fit_bounds([sw,ne])
week_map

In [ ]:
holi_map = folium.Map(location=map_center,tiles='openstreetmap',zoomSnap=0.5)
HeatMap(subdata1_holi,radius=14).add_to(holi_map)
holi_map.fit_bounds([sw,ne])
holi_map

In [ ]:
# Let's study the decelaration distribution by weekday/holiday

# Convert the deceleration value into cotegories
bins = [0.25,0.3,0.4,1]
impacts_name = ['Risk 0.2+','Risk 0.3+','Risk 0.4+']
data['Deceleration Category'] = pd.cut(data['Deceleration'],bins=bins,labels=impacts_name,right=False)


subdata2 = data[['Weekday/Holiday','Deceleration','Deceleration Category']]
subdata2 = subdata2.groupby(['Weekday/Holiday','Deceleration Category'],observed=True).count()
subdata2 = subdata2.unstack(level=0)
subdata2

In [ ]:
subdata2.plot(kind='pie',subplots=True,autopct='%1.1f%%',figsize=(12,5))
plt.suptitle('Deceleration distribution on weekdays and holidays')
plt.legend(loc='upper right')

In [ ]:
# Now let's make heatmaps according to the period of the day
# Convert the deceleration value into cotegories
time_bins = pd.to_datetime(['00:00:00','06:00:00','12:00:00','18:00:00','23:59:59'], format='%H:%M:%S').time
period_labels = ['Night','Morning','Afternoon','Evening']
data['Day Period'] = pd.cut(data['Time'],bins=time_bins,labels=period_labels,right=False)

subdata3 = data[['Latitude','Longitude','Day Period']]
night_data3 = subdata3[subdata3['Day Period'] == 'Night'].drop('Day Period',axis=1)
morn_data3 = subdata3[subdata3['Day Period'] == 'Morning'].drop('Day Period',axis=1)
aftn_data3 = subdata3[subdata3['Day Period'] == 'Afternoon'].drop('Day Period',axis=1)
even_data3 = subdata3[subdata3['Day Period'] == 'Evening'].drop('Day Period',axis=1)

In [ ]:
night_map = folium.Map(location=map_center,tiles='openstreetmap',zoomSnap=0.5)
HeatMap(night_data3,radius=14).add_to(night_map)
night_map.fit_bounds([sw,ne])
night_map

In [ ]:
morn_map = folium.Map(location=map_center,tiles='openstreetmap',zoomSnap=0.5)
HeatMap(morn_data3,radius=14).add_to(morn_map)
morn_map.fit_bounds([sw,ne])
morn_map

In [ ]:
aftn_map = folium.Map(location=map_center,tiles='openstreetmap',zoomSnap=0.5)
HeatMap(aftn_data3,radius=14).add_to(aftn_map)
aftn_map.fit_bounds([sw,ne])
aftn_map

In [ ]:
even_map = folium.Map(location=map_center,tiles='openstreetmap',zoomSnap=0.5)
HeatMap(even_data3,radius=14).add_to(even_map)
even_map.fit_bounds([sw,ne])
even_map

In [ ]:
# Now we will study the distibution of deceleration through period of the day
subdata4 = data[['Time','Deceleration']]
subdata4 = subdata4.groupby('Time').count()
subdata4.info

In [ ]:
ax = subdata4.plot.bar(y='Deceleration',figsize=(10, 6))
#ax.xaxis.set_major_locator(mdates.HourLocator(interval=1))
#ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
#plt.xticks(rotation=45, ha='right')
ax.set_xticks([])
plt.title('Deceleration distribution through time of the day')
ax.grid(True)
ax.set_ylabel('Number of deceleration')

In [ ]:
# Now we will study the distibution of types through period of the day
subdata5 = data[['Day Period','Deceleration Category','Deceleration']]
subdata5 = subdata5.groupby(['Day Period','Deceleration Category'],observed=True).count()
subdata5 = subdata5.unstack(level=1)
subdata5

In [ ]:
subdata5.plot(kind='bar',figsize=(12,6))
plt.suptitle('Deceleration distribution by period of the day')